## Data Pre-processing

In [207]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import timedelta

In [66]:
df = pd.read_json('transactions.json',lines=True,orient='records')

In [67]:
# Replacing null values with NaN
df = df.replace(r'^\s*$', np.nan, regex=True)

In [68]:
# Dropping with features with zero entries
df.drop(['echoBuffer','merchantCity','merchantState','merchantZip','posOnPremises','recurringAuthInd'],
                  axis=1,inplace=True)

In [69]:
# Creating Date and Time Column
df['Date']=df['transactionDateTime'].apply(lambda x: x.split('T')[0])
df['Time']=df['transactionDateTime'].apply(lambda x: x.split('T')[1])

In [70]:
def concat(x):
    y=str('T') + str(x)
    return y
df.reset_index(drop=False,inplace=True)
df['transactionKey']=df['index'].apply(lambda x: concat(x))
df.drop(['index'],axis=1,inplace=True)

In [71]:
df.head()

,accountNumber,customerId,creditLimit,availableMoney,transactionDateTime,transactionAmount,merchantName,acqCountry,merchantCountryCode,posEntryMode,...,enteredCVV,cardLast4Digits,transactionType,currentBalance,cardPresent,expirationDateKeyInMatch,isFraud,Date,Time,transactionKey
0,737265056,737265056,5000,5000.0,2016-08-13T14:27:32,98.55,Uber,US,US,02,...,414,1803,PURCHASE,0.0,False,False,False,2016-08-13,14:27:32,T0
1,737265056,737265056,5000,5000.0,2016-10-11T05:05:54,74.51,AMC #191138,US,US,09,...,486,767,PURCHASE,0.0,True,False,False,2016-10-11,05:05:54,T1
2,737265056,737265056,5000,5000.0,2016-11-08T09:18:39,7.47,Play Store,US,US,09,...,486,767,PURCHASE,0.0,False,False,False,2016-11-08,09:18:39,T2
3,737265056,737265056,5000,5000.0,2016-12-10T02:14:50,7.47,Play Store,US,US,09,...,486,767,PURCHASE,0.0,False,False,False,2016-12-10,02:14:50,T3
4,830329091,830329091,5000,5000.0,2016-03-24T21:04:46,71.18,Tim Hortons #947751,US,US,02,...,885,3143,PURCHASE,0.0,True,False,False,2016-03-24,21:04:46,T4


## Identifying Reversed transactions

In [76]:
df['transactionType'].isnull().sum()

698

In [77]:
#Replacing missing values in transactionType as 'PURCHASE'
df['transactionType']=df['transactionType'].fillna(value='PURCHASE')

In [78]:
# Creating Timestamp column
df.Date=pd.to_datetime(df.Date)
df['Timestamp']=pd.to_datetime(df.Date.astype(str)+' '+df.Time.astype(str))

In [98]:
# Sorting data in ascending order with respect to customerId, merchantName, Timestamp
data=df[['customerId','merchantName','transactionAmount','transactionType','transactionKey','Timestamp']]
data=data.sort_values(by=['customerId','merchantName','Timestamp'],ascending=True)

In [99]:
data.head()

,customerId,merchantName,transactionAmount,transactionType,transactionKey,Timestamp
541917,100088067,1st BBQ,200.12,PURCHASE,T541917,2016-04-08 19:56:36
541962,100088067,1st Pub,95.11,PURCHASE,T541962,2016-11-09 07:26:06
541920,100088067,1st Sandwitch Bar #758805,5.26,PURCHASE,T541920,2016-05-06 02:54:01
541904,100088067,34th BBQ #412895,141.60,PURCHASE,T541904,2016-02-03 04:48:54
541925,100088067,AMC #191138,304.15,PURCHASE,T541925,2016-06-03 18:50:55


In [100]:
# Removing transactions with ADDRESS_VERIFICATION type since, there transaction amount is $0
data=data.reset_index(drop=True)
data=data[-(data.transactionType=='ADDRESS_VERIFICATION')]
data=data.reset_index(drop=True)

#### Considering the purchase and reversal transaction are consecutive

In [107]:
reverse = data.loc[data['transactionType']=='REVERSAL']
reverse.head()

,customerId,merchantName,transactionAmount,transactionType,transactionKey,Timestamp
13,100088067,AMC #79863,22.32,REVERSAL,T541964,2016-11-20 08:00:04
124,100328049,Lyft,43.74,REVERSAL,T87920,2016-01-15 20:36:18
140,100328049,McDonalds #423357,284.97,REVERSAL,T87936,2016-03-26 17:35:09
263,100737756,34th BBQ #436606,93.67,REVERSAL,T151499,2016-06-06 22:56:52
278,100737756,Best Bistro #262998,501.29,REVERSAL,T151541,2016-08-24 20:21:19


In [110]:
purchase_index = reverse.index.values.astype(int)-1

In [113]:
purchase = data.iloc[purchase_index]
purchase.head()

,customerId,merchantName,transactionAmount,transactionType,transactionKey,Timestamp
12,100088067,AMC #79863,22.32,PURCHASE,T541963,2016-11-20 07:57:05
123,100328049,Lyft,43.74,PURCHASE,T87919,2016-01-15 20:34:35
139,100328049,McDonalds #423357,284.97,PURCHASE,T87935,2016-03-24 22:57:15
262,100737756,34th BBQ #436606,93.67,PURCHASE,T151496,2016-05-30 00:42:32
277,100737756,Best Bistro #262998,501.29,PURCHASE,T151534,2016-08-11 09:20:41


In [118]:
purchase_index=pd.DataFrame(purchase_index)
purchase_index.columns=['Purchase_Index']

In [115]:
reverse_index=pd.DataFrame(reverse_index)
reverse_index.columns=['Reverse_Index']

In [116]:
purchase_amount=pd.DataFrame(purchase['transactionAmount'].values)
purchase_amount.columns=['Purchase_Amount']

In [117]:
reverse_amount=pd.DataFrame(reverse['transactionAmount'].values)
reverse_amount.columns=['Reversal_Amount']

In [120]:
# Creating DataFrame to check reversal transactions
chk=pd.concat([purchase_index,reverse_index,purchase_amount,reverse_amount],axis=1) 
chk.head()

,Purchase_Index,Reverse_Index,Purchase_Amount,Reversal_Amount
0,12,13,22.32,22.32
1,123,124,43.74,43.74
2,139,140,284.97,284.97
3,262,263,93.67,93.67
4,277,278,501.29,501.29


In [122]:
chk['new'] = np.where((chk['Purchase_Amount'] == chk['Reversal_Amount']), True, False)

In [124]:
chk.head()

,Purchase_Index,Reverse_Index,Purchase_Amount,Reversal_Amount,new
0,12,13,22.32,22.32,True
1,123,124,43.74,43.74,True
2,139,140,284.97,284.97,True
3,262,263,93.67,93.67,True
4,277,278,501.29,501.29,True


In [125]:
chk.new.value_counts()

True     12665
False     7638
Name: new, dtype: int64

#### By checking the consecutive transactions, I was able to track 12665 reversal transactions.

In [127]:
tracked = chk.loc[chk['new']==True]

#### Creating DataFrame for all tracked reversal transactions

In [135]:
track_index=np.concatenate((tracked['Purchase_Index'], tracked['Reverse_Index']))
track_reverse=data.iloc[track_index]

In [137]:
track_reverse=track_reverse.sort_values(by=['customerId','merchantName','Timestamp'],ascending=True)
track_reverse.head()

,customerId,merchantName,transactionAmount,transactionType,transactionKey,Timestamp
12,100088067,AMC #79863,22.32,PURCHASE,T541963,2016-11-20 07:57:05
13,100088067,AMC #79863,22.32,REVERSAL,T541964,2016-11-20 08:00:04
123,100328049,Lyft,43.74,PURCHASE,T87919,2016-01-15 20:34:35
124,100328049,Lyft,43.74,REVERSAL,T87920,2016-01-15 20:36:18
139,100328049,McDonalds #423357,284.97,PURCHASE,T87935,2016-03-24 22:57:15


In [140]:
track_reverse.transactionType.value_counts()

REVERSAL    12665
PURCHASE    12665
Name: transactionType, dtype: int64

#### I was able to track 12665 reverse transactions out of the 20303 transactions

In [141]:
track_reverse.groupby(['transactionType']).sum()['transactionAmount']

transactionType
PURCHASE    1907433.62
REVERSAL    1907433.62
Name: transactionAmount, dtype: float64

In [143]:
reverse.groupby(['transactionType']).sum()['transactionAmount']

transactionType
REVERSAL    2821792.5
Name: transactionAmount, dtype: float64

#### The transaction amount of tracked reversal transactions is  $1907433.62

#### The overall transaction amount of reverse transactions is  $2821792.5

#### 7638 transactions were not tracked with transaction amount of $914,358.88

## Identifying Multi-swipe transactions

#### To identify multi-swipes, I am initially sorting the DataFrame in ascending order with respect to cusromerId, merchantName and Timestamp. The sorted data is available from previous. Then, to identify the multi-swipe transaction, I subset data with all the duplicate transactions and then classify the multi-swipe transactions with time difference less than 180 seconds.

In [248]:
# Removing Reverse transactions
mst=data.reset_index(drop=True)
mst=mst[-(data.transactionType=='REVERSAL')]
mst=mst.reset_index(drop=True)

In [249]:
mst.head()

,customerId,merchantName,transactionAmount,transactionType,transactionKey,Timestamp
0,100088067,1st BBQ,200.12,PURCHASE,T541917,2016-04-08 19:56:36
1,100088067,1st Pub,95.11,PURCHASE,T541962,2016-11-09 07:26:06
2,100088067,1st Sandwitch Bar #758805,5.26,PURCHASE,T541920,2016-05-06 02:54:01
3,100088067,34th BBQ #412895,141.60,PURCHASE,T541904,2016-02-03 04:48:54
4,100088067,AMC #191138,304.15,PURCHASE,T541925,2016-06-03 18:50:55


In [247]:
k=mst.loc[:,['customerId','merchantName','transactionAmount','transactionType']]

In [250]:
k.head()

,customerId,merchantName,transactionAmount,transactionType
0,100088067,1st BBQ,200.12,PURCHASE
1,100088067,1st Pub,95.11,PURCHASE
2,100088067,1st Sandwitch Bar #758805,5.26,PURCHASE
3,100088067,34th BBQ #412895,141.60,PURCHASE
4,100088067,AMC #191138,304.15,PURCHASE


In [251]:
# Finding duplicate transactions
duplicate = k.duplicated(keep=False)

In [252]:
Duplicate=pd.DataFrame(duplicate)
Duplicate.columns=['Duplicate']

In [253]:
a=pd.concat([mst,Duplicate],axis=1) 
a.head()

,customerId,merchantName,transactionAmount,transactionType,transactionKey,Timestamp,Duplicate
0,100088067,1st BBQ,200.12,PURCHASE,T541917,2016-04-08 19:56:36,False
1,100088067,1st Pub,95.11,PURCHASE,T541962,2016-11-09 07:26:06,False
2,100088067,1st Sandwitch Bar #758805,5.26,PURCHASE,T541920,2016-05-06 02:54:01,False
3,100088067,34th BBQ #412895,141.60,PURCHASE,T541904,2016-02-03 04:48:54,False
4,100088067,AMC #191138,304.15,PURCHASE,T541925,2016-06-03 18:50:55,False


In [254]:
# Subsetting the data with duplicate transactions only
b = a.loc[a['Duplicate']==True]

In [255]:
b.head()

,customerId,merchantName,transactionAmount,transactionType,transactionKey,Timestamp,Duplicate
13,100088067,Apple iTunes,3.96,PURCHASE,T541919,2016-05-05 22:47:50,True
14,100088067,Apple iTunes,3.96,PURCHASE,T541926,2016-06-05 22:10:41,True
15,100088067,Apple iTunes,3.96,PURCHASE,T541929,2016-07-05 22:22:49,True
16,100088067,Apple iTunes,3.96,PURCHASE,T541936,2016-08-05 00:55:48,True
17,100088067,Apple iTunes,3.96,PURCHASE,T541945,2016-09-05 13:14:22,True


In [256]:
# Calculating time difference between the transactions
b['difference']=b.Timestamp.diff()

<ipython-input-256-0cd736aaa625>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  b['difference']=b.Timestamp.diff()


In [257]:
b.head()

,customerId,merchantName,transactionAmount,transactionType,transactionKey,Timestamp,Duplicate,difference
13,100088067,Apple iTunes,3.96,PURCHASE,T541919,2016-05-05 22:47:50,True,NaT
14,100088067,Apple iTunes,3.96,PURCHASE,T541926,2016-06-05 22:10:41,True,30 days 23:22:51
15,100088067,Apple iTunes,3.96,PURCHASE,T541929,2016-07-05 22:22:49,True,30 days 00:12:08
16,100088067,Apple iTunes,3.96,PURCHASE,T541936,2016-08-05 00:55:48,True,30 days 02:32:59
17,100088067,Apple iTunes,3.96,PURCHASE,T541945,2016-09-05 13:14:22,True,31 days 12:18:34


In [258]:
z = timedelta(0,0)
z
td = timedelta(0,180)
td

datetime.timedelta(seconds=180)

In [259]:
# Checking for timedifference less than 180 seconds
b['multi_swipe']=(b['difference']<td) & (b['difference']>z)

<ipython-input-259-4e071ba3d68d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  b['multi_swipe']=(b['difference']<td) & (b['difference']>z)


In [264]:
#Sub-setting data with multi-swipe transactions only
multi_swipe = b.loc[b['multi_swipe']==True]

In [261]:
multi_swipe.head()

,customerId,merchantName,transactionAmount,transactionType,transactionKey,Timestamp,Duplicate,difference,multi_swipe
44,100088067,Fresh Flowers,411.35,PURCHASE,T541956,2016-10-16 18:01:02,True,0 days 00:00:02,True
251,100737756,34th BBQ #166379,43.25,PURCHASE,T151518,2016-07-10 14:32:06,True,0 days 00:00:59,True
318,100737756,Franks Deli,693.50,PURCHASE,T151427,2016-01-18 01:55:28,True,0 days 00:00:04,True
319,100737756,Franks Deli,693.50,PURCHASE,T151428,2016-01-18 01:58:26,True,0 days 00:02:58,True
374,100737756,South Steakhouse #73819,211.22,PURCHASE,T151513,2016-07-02 12:07:00,True,0 days 00:01:56,True


In [262]:
multi_swipe.transactionType.value_counts()

PURCHASE    7474
Name: transactionType, dtype: int64

In [263]:
# Calculating total mutli-swipe transaction amount leaving the first transaction
multi_swipe.sum()['transactionAmount']

1105654.5899999999

#### Total value of multi-swipe is 7474
#### Total transaction amount of multi-swipe transactions is $1105654